### This notebook includes the experiments for cross validation.

- Standing agent is P1, we are learning P2 (P2 is human player).

- Some frames will be removed, there are frames that;
    - Both players standing.
    - Consecutive ones(If frame's action is same with previous frame, it will be removed).
    - If we are(P2) in RECOV frame(Since we don't make RECOV frames ourselves).

### Importing libraries

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import TensorBoard
import time
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import json

# For using core package that located in the two upper folder.
import os,sys
sys.path.append('../../')

from core.json_importer import parse_json_file, parse_all_files
from core.filters import remove_both_standing_frames, remove_same_consecutive_actions, remove_recov_frames
from core.actions import one_hot_encode, decode
from core.preproccessing import Normalizer
from core.helpers import write_file

Using TensorFlow backend.


In [3]:
# Create out file folder if not exists
OUT_FOLDER = 'generated_files'
if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

In [8]:
# Open files
training_data = parse_all_files("/Users/oztiryakimeric/Dropbox/bulk_train_results/data/bcp/train")

### Pre process data

- In the pre process phase, we remove P2's datas from training set in order to crate labels. Labels are just actions of P2.

- We make actions one-hot encoding. 

- One-hot encoding for inputs and labels are not same. For example, Dash action in the input may have encoding [0,1] while Dash action in the label have encoding [0, 0, 1, 0].

- Integer values normalized. (P1-HP, P2-HP, P1-X, P1-Y, P2-X, P2-Y)

- After pre-process we just have <font color='red'>[P1-Action, P1-HP, P2-HP, P1-X, P1-Y, P2-X, P2-Y] -> P2-Action</font>

#### Remove unneeded frames

In [9]:
# Apply filters for training data
tr_deleted = training_data.filter(remove_both_standing_frames)
tr_deleted += training_data.filter(remove_recov_frames)
tr_deleted += training_data.filter(remove_same_consecutive_actions)


print('There are {} train data.({} deleted.)'.format(len(training_data), tr_deleted))

There are 8242 train data.(477758 deleted.)


#### Encoding and normalization

In [11]:
def normalize_and_save(data, file_name):
    p1_hp_normalizer = Normalizer()
    p1_normalized_hp = p1_hp_normalizer.normalize(data)
    p1_hp_normalizer.save(OUT_FOLDER + '/' + file_name)
    return p1_normalized_hp

def process_data(game_data_obj):
    ## Pre process data
    processed_data = []

    # Create one hot encoding for actions (For input and labels)
    p1_one_hot_encoded_actions = one_hot_encode(game_data_obj.get_column("P1-action"))
    labels = one_hot_encode(game_data_obj.get_column("P2-action"))

    normalized_xp1_distance = normalize_and_save([frame["P1-x"] for frame in game_data_obj], "xp1_norm.save")
    normalized_xp2_distance = normalize_and_save([frame["P2-x"] for frame in game_data_obj], "xp2_norm.save")
    normalized_yp1_distance = normalize_and_save([frame["P1-y"] for frame in game_data_obj], "yp1_norm.save")
    normalized_yp2_distance = normalize_and_save([frame["P2-y"] for frame in game_data_obj], "yp2_norm.save")

    for index in range(len(game_data_obj)):    
        processed_row = []
        processed_row.extend(p1_one_hot_encoded_actions[index])
        processed_row.extend(normalized_xp1_distance[index])
        processed_row.extend(normalized_xp2_distance[index])
        processed_row.extend(normalized_yp1_distance[index])
        processed_row.extend(normalized_yp2_distance[index])
        processed_data.append(processed_row)
    processed_data = np.array(processed_data)
    labels = np.array(labels)
    
    return processed_data, labels
   
tr_data, tr_labels = process_data(training_data)

### Example data

In [6]:
EXAMPLE_ROW = 10

print("There are %d frames in dataset." % len(tr_data))
print("After pre processing the shape of our dataset is %s" % str(tr_data.shape))
print("\nOne example in index %d." % EXAMPLE_ROW)

row = tr_data[EXAMPLE_ROW]
print("\tProcessed Frame:" )
print("\t\tP1 Action(one-hot) Shape:\t[1, %d]" % 54)
print("\t\tP1 Hp(norm.):\t\t\t%s" % row[55])
print("\t\tP2 Hp(norm.):\t\t\t%s" % row[56])
print("\t\tX Dist(norm.):\t\t\t%s" % row[57])
print("\t\tY Dist(norm.):\t\t\t%s" % row[58])
print("\t\tLabel Shape:\t\t\t[1, %d]" % len(tr_labels[EXAMPLE_ROW]))

There are 11428 frames in dataset.
After pre processing the shape of our dataset is (11428, 61)

One example in index 10.
	Processed Frame:
		P1 Action(one-hot) Shape:	[1, 54]
		P1 Hp(norm.):			0.01594896331738437
		P2 Hp(norm.):			0.010683760683760684
		X Dist(norm.):			0.0425
		Y Dist(norm.):			0.08625000000000001
		Label Shape:			[1, 55]


### Neural Network Design

Our neural network has two hidden layers in this test. They has 12 and 8 neurons respectively.

In [7]:
# Constants
INPUT_LAYER_SIZE = tr_data.shape[1]
OUTPUT_LAYER_SIZE = tr_labels.shape[1]

In [8]:
def create_model(neuron_count):
    model = Sequential()
    model.add(Dense(neuron_count, input_dim=INPUT_LAYER_SIZE, activation='relu'))
    model.add(Dense(OUTPUT_LAYER_SIZE, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [11]:
COMPLEXITIES = [10, 25, 50, 75, 100, 175, 250, 350, 500, 750, 1000, 1500, 2500]
EPOCH = 500
BATCH = 32

In [ ]:
if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

# Create session folder.
SESSION_FOLDER = OUT_FOLDER + '/session_{}'.format(time.time())

os.makedirs(SESSION_FOLDER)
os.makedirs(SESSION_FOLDER + '/models')
os.makedirs(SESSION_FOLDER + '/logs')
os.makedirs(SESSION_FOLDER + '/histories')

histories = []
# Create and run networks
for complexity in COMPLEXITIES:
    starting_time = time.clock()
    file_name = str(complexity)
    
    model = create_model(complexity)
    print('Model created with {} neurons.'.format(complexity))
    
    # Logging for tensorboard
    tensorboard = TensorBoard(log_dir= SESSION_FOLDER + "/logs/log_{}".format(file_name))
    
    print('\tTraining started.')
    history = model.fit(tr_data, tr_labels, epochs=EPOCH, validation_split=0.2,
                    shuffle=True, batch_size=BATCH, callbacks=[tensorboard])
    print('\tTraining ended.')
    
    # Display result
    scores = model.evaluate(te_data, te_labels)
    print("\t\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

    time_taken = (time.clock() - starting_time) * 1000.0
    
    model_info = {
        'info': 'One hidden layer ANN with {} neurons'.format(complexity),
        'epoch': EPOCH,
        'batch': BATCH,
        'time_taken': time_taken
    }
    
    # Save model and history
    model.save(SESSION_FOLDER + '/models/model_{}.h5'.format(file_name))
    write_file(SESSION_FOLDER + '/models/config_{}.json'.format(file_name), model.to_json())
    write_file(SESSION_FOLDER + '/models/info_{}.json'.format(file_name), json.dumps(model_info))
    
    pickle.dump(history, open(SESSION_FOLDER + '/histories/history_{}.save'.format(file_name), 'wb'))
    
    histories.append(history)
    
    
print('Session completed successfully. {} networks created and their data saved.'.format(len(COMPLEXITIES)))

Model created with 10 neurons.
	Training started.
Train on 9142 samples, validate on 2286 samples
Epoch 1/500
9142/9142 [==============================] - 1s 94us/step - loss: 3.2959 - acc: 0.1551 - val_loss: 2.7375 - val_acc: 0.1802
Epoch 2/500
9142/9142 [==============================] - 1s 59us/step - loss: 2.6958 - acc: 0.2343 - val_loss: 2.5893 - val_acc: 0.2734
Epoch 3/500
9142/9142 [==============================] - 1s 75us/step - loss: 2.5951 - acc: 0.2688 - val_loss: 2.5004 - val_acc: 0.2843
Epoch 4/500
9142/9142 [==============================] - 1s 83us/step - loss: 2.5063 - acc: 0.2972 - val_loss: 2.4148 - val_acc: 0.3163
Epoch 5/500
9142/9142 [==============================] - 1s 73us/step - loss: 2.4300 - acc: 0.3117 - val_loss: 2.3539 - val_acc: 0.3180
Epoch 6/500
9142/9142 [==============================] - 1s 67us/step - loss: 2.3749 - acc: 0.3144 - val_loss: 2.3080 - val_acc: 0.3220
Epoch 7/500
9142/9142 [==============================] - 1s 66us/step - loss: 2.3341 -

Epoch 120/500
9142/9142 [==============================] - 0s 54us/step - loss: 1.8998 - acc: 0.3984 - val_loss: 1.9140 - val_acc: 0.4024
Epoch 121/500
9142/9142 [==============================] - 0s 52us/step - loss: 1.8985 - acc: 0.4007 - val_loss: 1.9175 - val_acc: 0.4038
Epoch 122/500
9142/9142 [==============================] - 0s 51us/step - loss: 1.8968 - acc: 0.4025 - val_loss: 1.9127 - val_acc: 0.4033
Epoch 123/500
9142/9142 [==============================] - 0s 51us/step - loss: 1.8965 - acc: 0.4007 - val_loss: 1.9118 - val_acc: 0.4011
Epoch 124/500
9142/9142 [==============================] - 0s 50us/step - loss: 1.8948 - acc: 0.4008 - val_loss: 1.9144 - val_acc: 0.4059
Epoch 125/500
9142/9142 [==============================] - 0s 52us/step - loss: 1.8938 - acc: 0.4005 - val_loss: 1.9088 - val_acc: 0.4059
Epoch 126/500
9142/9142 [==============================] - 0s 51us/step - loss: 1.8927 - acc: 0.4018 - val_loss: 1.9084 - val_acc: 0.4068
Epoch 127/500
9142/9142 [=========

9142/9142 [==============================] - 1s 55us/step - loss: 1.8025 - acc: 0.4188 - val_loss: 1.8161 - val_acc: 0.4265
Epoch 239/500
9142/9142 [==============================] - 1s 58us/step - loss: 1.8010 - acc: 0.4158 - val_loss: 1.8192 - val_acc: 0.4234
Epoch 240/500
9142/9142 [==============================] - 1s 57us/step - loss: 1.8013 - acc: 0.4204 - val_loss: 1.8171 - val_acc: 0.4186
Epoch 241/500
9142/9142 [==============================] - 0s 53us/step - loss: 1.8006 - acc: 0.4189 - val_loss: 1.8150 - val_acc: 0.4230
Epoch 242/500
9142/9142 [==============================] - 1s 56us/step - loss: 1.8003 - acc: 0.4174 - val_loss: 1.8157 - val_acc: 0.4182
Epoch 243/500
9142/9142 [==============================] - 0s 53us/step - loss: 1.7988 - acc: 0.4194 - val_loss: 1.8172 - val_acc: 0.4252
Epoch 244/500
9142/9142 [==============================] - 0s 51us/step - loss: 1.8001 - acc: 0.4197 - val_loss: 1.8183 - val_acc: 0.4204
Epoch 245/500
9142/9142 [=======================

9142/9142 [==============================] - 1s 65us/step - loss: 1.7654 - acc: 0.4215 - val_loss: 1.7840 - val_acc: 0.4234
Epoch 357/500
9142/9142 [==============================] - 1s 78us/step - loss: 1.7648 - acc: 0.4208 - val_loss: 1.7864 - val_acc: 0.4226
Epoch 358/500
9142/9142 [==============================] - 1s 61us/step - loss: 1.7645 - acc: 0.4216 - val_loss: 1.7820 - val_acc: 0.4361
Epoch 359/500
9142/9142 [==============================] - 1s 60us/step - loss: 1.7660 - acc: 0.4239 - val_loss: 1.7824 - val_acc: 0.4265
Epoch 360/500
9142/9142 [==============================] - 1s 60us/step - loss: 1.7654 - acc: 0.4245 - val_loss: 1.7859 - val_acc: 0.4274
Epoch 361/500
9142/9142 [==============================] - 1s 60us/step - loss: 1.7647 - acc: 0.4211 - val_loss: 1.7849 - val_acc: 0.4213
Epoch 362/500
9142/9142 [==============================] - 1s 63us/step - loss: 1.7648 - acc: 0.4195 - val_loss: 1.7867 - val_acc: 0.4313
Epoch 363/500
9142/9142 [=======================

9142/9142 [==============================] - 1s 58us/step - loss: 1.7505 - acc: 0.4234 - val_loss: 1.7760 - val_acc: 0.4248
Epoch 475/500
9142/9142 [==============================] - 0s 53us/step - loss: 1.7502 - acc: 0.4252 - val_loss: 1.7738 - val_acc: 0.4248
Epoch 476/500
9142/9142 [==============================] - 1s 55us/step - loss: 1.7508 - acc: 0.4227 - val_loss: 1.7756 - val_acc: 0.4304
Epoch 477/500
9142/9142 [==============================] - 1s 59us/step - loss: 1.7512 - acc: 0.4229 - val_loss: 1.7746 - val_acc: 0.4261
Epoch 478/500
9142/9142 [==============================] - 0s 54us/step - loss: 1.7506 - acc: 0.4236 - val_loss: 1.7756 - val_acc: 0.4226
Epoch 479/500
9142/9142 [==============================] - 1s 58us/step - loss: 1.7500 - acc: 0.4231 - val_loss: 1.7773 - val_acc: 0.4265
Epoch 480/500
9142/9142 [==============================] - 1s 57us/step - loss: 1.7504 - acc: 0.4224 - val_loss: 1.7758 - val_acc: 0.4217
Epoch 481/500
9142/9142 [=======================

...

9142/9142 [==============================] - 1s 129us/step - loss: 0.9126 - acc: 0.6781 - val_loss: 1.8352 - val_acc: 0.5293
Epoch 221/500
9142/9142 [==============================] - 1s 131us/step - loss: 0.9102 - acc: 0.6810 - val_loss: 1.8668 - val_acc: 0.5267
Epoch 222/500
9142/9142 [==============================] - 1s 129us/step - loss: 0.9149 - acc: 0.6781 - val_loss: 1.8553 - val_acc: 0.5332
Epoch 223/500
9142/9142 [==============================] - 25s 3ms/step - loss: 0.9091 - acc: 0.6837 - val_loss: 1.8654 - val_acc: 0.5319
Epoch 224/500
9142/9142 [==============================] - 3s 301us/step - loss: 0.9109 - acc: 0.6816 - val_loss: 1.8273 - val_acc: 0.5389
Epoch 225/500
9142/9142 [==============================] - 2s 178us/step - loss: 0.9056 - acc: 0.6862 - val_loss: 1.8365 - val_acc: 0.5363
Epoch 226/500
9142/9142 [==============================] - 2s 166us/step - loss: 0.9114 - acc: 0.6774 - val_loss: 1.8620 - val_acc: 0.5398
Epoch 227/500
9142/9142 [=================

9142/9142 [==============================] - 1s 137us/step - loss: 0.8406 - acc: 0.7025 - val_loss: 1.9399 - val_acc: 0.5385
Epoch 280/500
9142/9142 [==============================] - 1s 141us/step - loss: 0.8381 - acc: 0.7040 - val_loss: 1.9661 - val_acc: 0.5280
Epoch 281/500
9142/9142 [==============================] - 1s 137us/step - loss: 0.8460 - acc: 0.7012 - val_loss: 1.9579 - val_acc: 0.5354
Epoch 282/500
9142/9142 [==============================] - 1s 139us/step - loss: 0.8440 - acc: 0.7048 - val_loss: 1.9415 - val_acc: 0.5372
Epoch 283/500
9142/9142 [==============================] - 1s 141us/step - loss: 0.8429 - acc: 0.7029 - val_loss: 1.9543 - val_acc: 0.5389
Epoch 284/500
9142/9142 [==============================] - 1s 146us/step - loss: 0.8396 - acc: 0.7002 - val_loss: 1.9797 - val_acc: 0.5258
Epoch 285/500
9142/9142 [==============================] - 1s 140us/step - loss: 0.8375 - acc: 0.7052 - val_loss: 2.0007 - val_acc: 0.5184
Epoch 286/500
9142/9142 [================

9142/9142 [==============================] - 1s 148us/step - loss: 0.7864 - acc: 0.7222 - val_loss: 2.0574 - val_acc: 0.5451
Epoch 339/500
9142/9142 [==============================] - 1s 142us/step - loss: 0.7923 - acc: 0.7235 - val_loss: 2.0812 - val_acc: 0.5350
Epoch 340/500
9142/9142 [==============================] - 1s 131us/step - loss: 0.7922 - acc: 0.7187 - val_loss: 2.0736 - val_acc: 0.5315
Epoch 341/500
9142/9142 [==============================] - 1s 132us/step - loss: 0.7880 - acc: 0.7167 - val_loss: 2.0826 - val_acc: 0.5341
Epoch 342/500
9142/9142 [==============================] - 1s 132us/step - loss: 0.7891 - acc: 0.7188 - val_loss: 2.0717 - val_acc: 0.5284
Epoch 343/500
9142/9142 [==============================] - 1s 132us/step - loss: 0.7808 - acc: 0.7236 - val_loss: 2.0985 - val_acc: 0.5328
Epoch 344/500
9142/9142 [==============================] - 1s 131us/step - loss: 0.7827 - acc: 0.7179 - val_loss: 2.0825 - val_acc: 0.5306
Epoch 345/500
9142/9142 [================

Epoch 397/500
9142/9142 [==============================] - 1s 160us/step - loss: 0.7452 - acc: 0.7339 - val_loss: 2.1579 - val_acc: 0.5407
Epoch 398/500
9142/9142 [==============================] - 1s 159us/step - loss: 0.7392 - acc: 0.7349 - val_loss: 2.1616 - val_acc: 0.5337
Epoch 399/500
9142/9142 [==============================] - 1s 149us/step - loss: 0.7423 - acc: 0.7358 - val_loss: 2.1747 - val_acc: 0.5324
Epoch 400/500
9142/9142 [==============================] - 1s 144us/step - loss: 0.7439 - acc: 0.7349 - val_loss: 2.1544 - val_acc: 0.5459
Epoch 401/500
9142/9142 [==============================] - 1s 143us/step - loss: 0.7374 - acc: 0.7363 - val_loss: 2.1978 - val_acc: 0.5324
Epoch 402/500
9142/9142 [==============================] - 1s 152us/step - loss: 0.7425 - acc: 0.7345 - val_loss: 2.1902 - val_acc: 0.5354
Epoch 403/500
9142/9142 [==============================] - 1s 159us/step - loss: 0.7347 - acc: 0.7334 - val_loss: 2.1641 - val_acc: 0.5354
Epoch 404/500
9142/9142 [==

Epoch 456/500
9142/9142 [==============================] - 1s 149us/step - loss: 0.7070 - acc: 0.7481 - val_loss: 2.2522 - val_acc: 0.5424
Epoch 457/500
9142/9142 [==============================] - 1s 154us/step - loss: 0.7085 - acc: 0.7431 - val_loss: 2.2572 - val_acc: 0.5424
Epoch 458/500
9142/9142 [==============================] - 1s 151us/step - loss: 0.7011 - acc: 0.7476 - val_loss: 2.2461 - val_acc: 0.5328
Epoch 459/500
9142/9142 [==============================] - 2s 170us/step - loss: 0.7032 - acc: 0.7441 - val_loss: 2.2678 - val_acc: 0.5319
Epoch 460/500
9142/9142 [==============================] - 1s 159us/step - loss: 0.7089 - acc: 0.7408 - val_loss: 2.2636 - val_acc: 0.5341
Epoch 461/500
9142/9142 [==============================] - 1s 162us/step - loss: 0.7063 - acc: 0.7426 - val_loss: 2.3721 - val_acc: 0.5184
Epoch 462/500
9142/9142 [==============================] - 1s 159us/step - loss: 0.7154 - acc: 0.7439 - val_loss: 2.3114 - val_acc: 0.5293
Epoch 463/500
9142/9142 [==